<center><h1>TP3 RCR2:</h1></center>
<center><h2>Réseaux Bayésiens</h2></center>
<center><h3>Construction d un poly-arbre</h3></center>

In [2]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianNetwork
#Etablissement de la structure
medical_model = BayesianNetwork([
    ('V','C'),
    ('A','C'),
    ('C','H'),
    ('A','S'),
])
#Définition des relations
vaccin_cpd= TabularCPD(
    variable='V',
    variable_card= 2,
    values=[[.6], [.4]]
)
adulte_cpd= TabularCPD(
    variable='A',
    variable_card= 2,
    values=[[.7], [.3]]
)
sportif_cpd= TabularCPD(
    variable='S',
    variable_card=2,
    values=[[.8,.9],
    [.2,.1]],
    evidence=['A'],
    evidence_card=[2]
)
covid_cpd= TabularCPD(
    variable='C',
    variable_card=2,
    values=[[.7,.8,.95,.8], [.3,.2,.05,.2]],
    evidence=['V','A'],
    evidence_card=[2,2]
)
hospital_cpd= TabularCPD(
    variable='H',
    variable_card=2,
    values=[[.9,.95], [.1,.05]],
    evidence=['C'],
    evidence_card=[2]
)
#on ajoute les relation à notre model
medical_model.add_cpds(vaccin_cpd,
sportif_cpd,covid_cpd,adulte_cpd,hospital_cpd)
#reverifier la structure de notre model
medical_model.get_cpds()

[<TabularCPD representing P(V:2) at 0x267fb77b340>,
 <TabularCPD representing P(S:2 | A:2) at 0x267fb77bfd0>,
 <TabularCPD representing P(C:2 | V:2, A:2) at 0x267fb78bfa0>,
 <TabularCPD representing P(A:2) at 0x267fb77b400>,
 <TabularCPD representing P(H:2 | C:2) at 0x267fb77a0d0>]

In [5]:
#trouver les chemins des noeuds pour determiner qu'est ce qui donne des infos sur un noeud
print(medical_model.active_trail_nodes('A'))
print()

print(medical_model.active_trail_nodes('H'))
print()

print(medical_model.active_trail_nodes('C'))

{'A': {'A', 'C', 'S', 'H'}}

{'H': {'A', 'V', 'C', 'S', 'H'}}

{'C': {'A', 'V', 'C', 'S', 'H'}}



In [6]:
#determinaison des independances locales
print(medical_model.local_independencies('S'))
print()

print(medical_model.local_independencies('H'))
print()

print(medical_model.local_independencies('A'))
print()

medical_model.get_independencies()

(S ⟂ C, H, V | A)

(H ⟂ A, S, V | C)

(A ⟂ V)



(A ⟂ V)
(A ⟂ H | C)
(A ⟂ V | S)
(A ⟂ H | C, S)
(A ⟂ H | C, V)
(A ⟂ H | C, S, V)
(V ⟂ A, S)
(V ⟂ H | C)
(V ⟂ A | S)
(V ⟂ S | A)
(V ⟂ H | C, S)
(V ⟂ S, H | C, A)
(V ⟂ S | A, H)
(V ⟂ H | C, S, A)
(V ⟂ S | C, H, A)
(C ⟂ S | A)
(C ⟂ S | A, H)
(C ⟂ S | A, V)
(C ⟂ S | A, H, V)
(S ⟂ V)
(S ⟂ H | C)
(S ⟂ C, H, V | A)
(S ⟂ H | C, V)
(S ⟂ H, V | C, A)
(S ⟂ C, H | A, V)
(S ⟂ C, V | A, H)
(S ⟂ H | A, C, V)
(S ⟂ V | C, H, A)
(S ⟂ C | A, H, V)
(H ⟂ A, S, V | C)
(H ⟂ S | A)
(H ⟂ A, S | C, V)
(H ⟂ A, V | C, S)
(H ⟂ S, V | C, A)
(H ⟂ S | A, V)
(H ⟂ A | C, S, V)
(H ⟂ S | A, C, V)
(H ⟂ V | C, S, A)

In [7]:
from pgmpy.inference import VariableElimination
medical_infer = VariableElimination(medical_model)
#calcul des proba existante pour vérifier que tout est correct
prob_vaccin = medical_infer.query(variables=['V'])
print(prob_vaccin)

+------+----------+
| V    |   phi(V) |
+======+==========+
| V(0) |   0.6000 |
+------+----------+
| V(1) |   0.4000 |
+------+----------+


In [8]:
#faire un autre calcul de proba
prob_hospital= medical_infer.query(variables=['H'])
print(prob_hospital)

+------+----------+
| H    |   phi(H) |
+======+==========+
| H(0) |   0.9100 |
+------+----------+
| H(1) |   0.0900 |
+------+----------+


In [10]:
#probabilité d'avoir le covid si on est sportif
proba_covid_sportif = medical_infer.query(variables=['C'], evidence= {'S':1}) 
print(proba_covid_sportif)

+------+----------+
| C    |   phi(C) |
+======+==========+
| C(0) |   0.8000 |
+------+----------+
| C(1) |   0.2000 |
+------+----------+


In [11]:
proba_hospital_adulteNonvaccin = medical_infer.query(variables=['H'], evidence= {'V':0, 'A':1}) 
print(proba_hospital_adulteNonvaccin)

+------+----------+
| H    |   phi(H) |
+======+==========+
| H(0) |   0.9100 |
+------+----------+
| H(1) |   0.0900 |
+------+----------+
